In [61]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sklearn.linear_model

In [62]:
oecd_bli = pd.read_csv("oecd_bli_2015.csv" , thousands= ',')
gdp_per_capita = pd.read_csv("gdp_per_capita.csv" , thousands= ',' , delimiter='\t' , encoding='latin1' , na_values= "n/a")

In [63]:
def prepare_country_stats(oecd_bli, gdp_per_capita):
    oecd_bli = oecd_bli[oecd_bli["INEQUALITY"]=="TOT"]
    oecd_bli = oecd_bli.pivot(index="Country", columns="Indicator", values="Value")
    gdp_per_capita.rename(columns={"2015": "GDP per capita"}, inplace=True)
    gdp_per_capita.set_index("Country", inplace=True)
    full_country_stats = pd.merge(left=oecd_bli, right=gdp_per_capita,
                                  left_index=True, right_index=True)
    full_country_stats.sort_values(by="GDP per capita", inplace=True)
    remove_indices = [0, 1, 6, 8, 33, 34, 35]
    keep_indices = list(set(range(36)) - set(remove_indices))
    return full_country_stats[["GDP per capita", 'Life satisfaction']].iloc[keep_indices]

In [64]:
country_stats = prepare_country_stats(oecd_bli , gdp_per_capita)
X = np.c_[country_stats["GDP per capita"]]
y = np.c_[country_stats["Life satisfaction"]]

In [65]:
country_stats.plot(kind = 'scatter' , x = "GDP per capita" , y = "Life satisfaction")
plt.show

In [66]:

model = sklearn.linear_model.LinearRegression()

# 모델 훈련
model.fit(X, y)

# 키프로스에 대한 예측
X_new = [[100000]]  # 키프로스 1인당 GDP
print(model.predict(X_new)) # 출력 [[ 5.96242338]]

In [67]:
import sklearn.neighbors

m = sklearn.neighbors.KNeighborsRegressor(n_neighbors= 10)

m.fit(X , y)
X_new = [[100000]]  # 키프로스 1인당 GDP
print(m.predict(X_new)) # 출력 [[ 5.96242338]]

In [68]:
import os
import tarfile
import urllib.request

DOWNLOAD_ROOT = "https://raw.githubusercontent.com/rickiepark/handson-ml2/master/"
HOUSING_PATH = os.path.join("datasets", "housing")
HOUSING_URL = DOWNLOAD_ROOT + "datasets/housing/housing.tgz"

def fetch_housing_data(housing_url=HOUSING_URL, housing_path=HOUSING_PATH):
    if not os.path.isdir(housing_path):
        os.makedirs(housing_path)
    tgz_path = os.path.join(housing_path, "housing.tgz")
    urllib.request.urlretrieve(housing_url, tgz_path)
    housing_tgz = tarfile.open(tgz_path)
    housing_tgz.extractall(path=housing_path)
    housing_tgz.close()
    

In [69]:
def load_housing_data(housing_path=HOUSING_PATH):
    csv_path = os.path.join("housing.csv")
    return pd.read_csv(csv_path)

In [70]:
housing = load_housing_data()
housing.head()

In [71]:
housing.info()

In [72]:
housing["longitude"].value_counts()

In [73]:
housing.hist(bins=50, figsize=(20,15))

In [74]:
from sklearn.model_selection import *

train_set , test_set = train_test_split(housing, test_size=0.2 , random_state=42)

In [75]:
housing["income_cat"] = pd.cut(housing["median_income"],
                               bins=[0., 1.5, 3.0, 4.5, 6., np.inf],
                               labels=[1, 2, 3, 4, 5])

housing["income_cat"].hist()

In [76]:
from sklearn.model_selection import StratifiedShuffleSplit

split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_index, test_index in split.split(housing, housing["income_cat"]):
    strat_train_set = housing.loc[train_index]
    strat_test_set = housing.loc[test_index]

In [77]:
strat_test_set["income_cat"].value_counts() / len(strat_test_set)

In [78]:
housing = strat_test_set.copy()
housing.plot(kind="scatter", x="longitude", y="latitude", alpha=0.1)

In [79]:
housing.plot(kind="scatter", x="longitude", y="latitude", alpha=0.4,
             s=housing["population"]/100, label="population", figsize=(10,7),
             c="median_house_value", cmap=plt.get_cmap("jet"), colorbar=True,
             sharex=False)
plt.legend()

In [83]:
images_path = os.path.join(PROJECT_ROOT_DIR, "images", "end_to_end_project")
os.makedirs(images_path, exist_ok=True)
DOWNLOAD_ROOT = "https://raw.githubusercontent.com/ageron/handson-ml2/master/"
filename = "california.png"
print("Downloading", filename)
url = DOWNLOAD_ROOT + "images/end_to_end_project/" + filename
urllib.request.urlretrieve(url, os.path.join(images_path, filename))